# Comprendre le séquençage des opérations du RAG
Ce notebook est mis à disposition afin de comprendre le fonctionnement du système de chat avec recherche de contexte.

### Étape 1 : Saisie de la question par l'utilisateur

In [10]:
# Libre à vous de changer la question si vous le souhaitez, mais tous les commentaires du notebook ont été rédigés en se basant sur une démonstration faite à partir de la question suivante :
query = "Quelle est la superficie de la région urbaine de Lyon ?"

### Étape 2 : Recherche de mots-clés dans la question de l'utilisateur
La recherche de mots-clés se fait au travers d'un modèle de NLP capable d'extraire, à partir d'une phrase donnée, les différentes entités qui figurent dedans (localité, organisation, personne …)

In [11]:
from src.ner import NaturalEntityRecognizer


ner = NaturalEntityRecognizer('Clemnt73/RoBERTa-ner')
keywords = ner(query)

keywords

['région urbaine de Lyon']

### Étape 3 : Recherche d'articles relatifs au sujet sur Wikipedia
Le modèle de reconnaissance d'entités est bien parvenu à isoler le(s) sujet(s) important(s) dans la phrase. Dès lors, nous pouvons effectuer une recherche sur Wikipedia à partir des mots-clés trouvés :

In [12]:
from typing import List

from src.wiki import search_wikipedia


wiki: List[str] = []

for keyword in keywords:
    wiki.append(search_wikipedia(keyword))
    
wiki

["La Région Urbaine de Lyon (RUL) était une association loi de 1901 qui réunissait les grandes collectivités locales de la région lyonnaise. Créée en 1989, elle avait pour missions d’animer la concertation entre ces collectivités, mais aussi d’élaborer des stratégies et de lancer des projets à l’échelle métropolitaine. Elle a cessé ses activités le 1er avril 2015.\n\n\n== Membres ==\nLa RUL réunissait en 2012 la Région Rhône-Alpes, le Pôle Métropolitain (qui rassemble le Grand Lyon et les communautés d'agglomération de Saint-Étienne Métropole, de ViennAgglo, et de la Porte de l'Isère), les départements de l’Ain, de l’Isère, de la Loire et du Rhône, les communautés d’agglomération de Villefranche-sur-Saône, du Roannais, et de Bourg-en-Bresse..\nElle était coprésidée par le président du Grand Lyon (Gérard Collomb) et le président de la Région Rhône-Alpes (Jean-Jack Queyranne). Le président délégué était Jean-Paul Bret, maire de Villeurbanne.\n\n\n== Objectifs ==\nLa région lyonnaise est 

La liste `wiki` contient le contenu du/des article(s) trouvé(s) sur Wikipédia à partir du/des mot(s) clés. Cet exemple nous montre que la liste contient un article ; mais en l'état, on ne sait toujours pas quelle partie de cet article sera la plus pertinente pour répondre à la question.

C'est ici qu'intervient la base de donnée vectorielle : par analyse sémantique, la base vectorielle sera en mesure de déterminer quelle partie de l'article est la *plus proche* de la question de l'utilisateur.

### Étape 4 : Recherche de l'information la plus pertinente parmi les articles trouvés
La première étape consiste à construire cette base de données vectorielle à partir de la liste `wiki` :

In [13]:
#TODO: Implémenter la partie construction de la base de données vectorielle

La seconde étape, maintenant que la base de données vectorielle est complète, est de faire une requête qui reprend la question initialement posée par l'utilisateur et retourne le texte le *plus proche* sémantiquement :

In [14]:
#TODO : Implémenter la partie requête de la base de données vectorielle
context = wiki[0]

### Étape 5 : Préparation du prompt
Nous avons maintenant obtenu l'information la plus pertinente par rapport à la question initialement posée. Il est donc désormais possible de préparer la requête qui sera par la suite envoyée au LLM. Pour cela, nous allons tout d'abord préparer un modèle générique de question :

In [15]:
PROMPT_TEMPLATE = """
Tu es un journaliste senior qui aime rétablir la vérité dans les informations.
Répond à la question en français et dit que tu ne sais pas si tu n'as pas l'information. Tu peux t'aider du contexte suivant, qui provient directement de Wikipedia, pour appuyer tes propos :

{context}

Question :

{question}
"""

Il s'agit ici d'un modèle qu'il faut compléter par les éléments que nous avons à notre disposition. En particulier, la variable `context` sera remplacée par l'article trouvé par la base de donnée vectorielle tandis que la variable `question` sera complétée par la question initialement posée par l'utilisateur :

In [16]:
prompt = PROMPT_TEMPLATE.format(
    context=context,
    question=query
)
print(prompt)


Tu es un journaliste senior qui aime rétablir la vérité dans les informations.
Répond à la question en français et dit que tu ne sais pas si tu n'as pas l'information. Tu peux t'aider du contexte suivant, qui provient directement de Wikipedia, pour appuyer tes propos :

La Région Urbaine de Lyon (RUL) était une association loi de 1901 qui réunissait les grandes collectivités locales de la région lyonnaise. Créée en 1989, elle avait pour missions d’animer la concertation entre ces collectivités, mais aussi d’élaborer des stratégies et de lancer des projets à l’échelle métropolitaine. Elle a cessé ses activités le 1er avril 2015.


== Membres ==
La RUL réunissait en 2012 la Région Rhône-Alpes, le Pôle Métropolitain (qui rassemble le Grand Lyon et les communautés d'agglomération de Saint-Étienne Métropole, de ViennAgglo, et de la Porte de l'Isère), les départements de l’Ain, de l’Isère, de la Loire et du Rhône, les communautés d’agglomération de Villefranche-sur-Saône, du Roannais, et de

### Étape 6 : Inférence
Notre prompt étant prêt, nous pouvons désormais procéder à l'inférence via l'API de *Groq*, qui utilise les LLM open sources pour répondre aux questions des utilisateurs. Mais avant cela, voyons ce qu'aurait répondu le LLM s'il n'avait pas eu le contexte qu'on lui a fourni :

In [17]:
from src.groq import get_groq_completions


print(get_groq_completions(query))

La région urbaine de Lyon, également appelée aire urbaine de Lyon, s'étend sur une superficie d'environ 5 628 km². Cela inclut l'unité urbaine de Lyon, qui couvre une superficie d'environ 478 km², ainsi que les communes périphériques qui forment la couronne périurbaine.

Il est important de noter que les définitions et les limites des aires urbaines peuvent varier selon les sources et les méthodologies utilisées. Par conséquent, les chiffres peuvent légèrement varier en fonction de la source de données.


Difficile de commenter la réponse, car elle différera à chaque nouvelle exécution du notebook … Toutefois, nous avons essayé plusieurs fois et n'avons jamais obtenu la bonne réponse : le LLM nous dit que la surface est de 5628 kilomètres-carrés alors que l'article *Wikipedia* fait mention d'une superficie de 10378 kilomètres-carrés.

Voyons maintenant comment diffère la réponse du LLM si on fournit les informations trouvées sur *Wikipedia* grâce aux précédentes étapes :

In [18]:
from src.groq import get_groq_completions


print(get_groq_completions(prompt))

La superficie de la Région Urbaine de Lyon est de 10 378 km2. Cette information est disponible dans la section "Périmètre" de l'article Wikipedia sur la Région Urbaine de Lyon.


Cette fois-ci, les informations sont exactes, et nous pouvons le vérifier par nous-même en allant sur l'article *Wikipedia* de Lyon ! Ce système permet ainsi aux utilisateurs, à travers une simple requête, de pouvoir accéder aux informations vérifiées par les contributeurs *Wikipedia* en quelques secondes seulement.

### Pour continuer …
Pour profiter d'une expérience utilisateur plus poussée, nous vous suggérons désormais de réitérer l'exercice en lançant le programme `main.py`, qui fournit, en plus de tout ce que nous avons exploré ici, une interface utilisateur à travers votre navigateur internet.